In [119]:
from imports import *

In [120]:
# Define names and filenames...

galaxy = 'ic5332'
galaxy_cat = galaxy
galaxy_hst = galaxy
root_dir = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies'
# Define names and filenames...

hstha_file = '%s/data_hstha/%s/hst_contsub/%s_hst_ha.fits' %(root_dir, galaxy_hst, galaxy_hst)
muscat_file = '%s/data_hstha/%s/muse/%s_nebmask.fits' %(root_dir, galaxy_hst, galaxy.upper())
musha_file = '%s/data_hstha/%s/muse/%s-*_MAPS.fits' %(root_dir, galaxy_hst, galaxy.upper())
musha_file = glob(musha_file)[0] #because of resolution in name
cutout_dir = '%s/data_hstha_nebulae_catalogue/%s/cutouts' %(root_dir, galaxy_hst)
dendro_dir = '%s/data_hstha_nebulae_catalogue/%s/catalogue' %(root_dir, galaxy_hst)
cutouts_hdus_dir = '%s/data_hstha_nebulae_catalogue/%s/cutouts_hdus' %(root_dir, galaxy_hst)
regions_file = '%s/sample.reg' %cutout_dir
regions_pickel_file = '%s/sample.pickel' %cutout_dir
sample_table_file = '%s/data_misc/sample_table/phangs_sample_table_v1p6.fits' %root_dir
muscat_table_file = '%s/data_misc/Nebulae_catalogue_v3/Nebulae_catalogue_v3.fits' %root_dir
catalog_mask_file = '%s/%s_mask.fits' %(dendro_dir, galaxy)

for prints in [hstha_file, muscat_file, musha_file, regions_file, regions_pickel_file, 
               sample_table_file, muscat_table_file, catalog_mask_file]:
    
    if os.path.isfile(prints):
        print('File found: %s' %prints)
    else: 
        print('File not found: %s' %prints)

# Define the cluster data... 
cluster_table_file_v_08pc = '%s/data_misc/multiscale_stellar_associations/%s/vselect/ws8pc/PHANGS_IR4_hst_wfc3_%s_v1p3_multi_assoc-vselect-ws8pc-main.fits' %(root_dir, galaxy_cat, galaxy_cat)
cluster_table_file_v_16pc = '%s/data_misc/multiscale_stellar_associations/%s/vselect/ws16pc/PHANGS_IR4_hst_wfc3_%s_v1p3_multi_assoc-vselect-ws16pc-main.fits' %(root_dir, galaxy_cat, galaxy_cat)
cluster_table_file_v_32pc = '%s/data_misc/multiscale_stellar_associations/%s/vselect/ws32pc/PHANGS_IR4_hst_wfc3_%s_v1p3_multi_assoc-vselect-ws32pc-main.fits' %(root_dir, galaxy_cat, galaxy_cat)
cluster_table_file_v_64pc = '%s/data_misc/multiscale_stellar_associations/%s/vselect/ws64pc/PHANGS_IR4_hst_wfc3_%s_v1p3_multi_assoc-vselect-ws64pc-main.fits' %(root_dir, galaxy_cat, galaxy_cat)

cluster_table_file_nuv_08pc = '%s/data_misc/multiscale_stellar_associations/%s/nuvselect/ws8pc/PHANGS_IR4_hst_wfc3_%s_v1p3_multi_assoc-nuvselect-ws8pc-main.fits' %(root_dir, galaxy_cat, galaxy_cat)
cluster_table_file_nuv_16pc = '%s/data_misc/multiscale_stellar_associations/%s/nuvselect/ws16pc/PHANGS_IR4_hst_wfc3_%s_v1p3_multi_assoc-nuvselect-ws16pc-main.fits' %(root_dir, galaxy_cat, galaxy_cat)
cluster_table_file_nuv_32pc = '%s/data_misc/multiscale_stellar_associations/%s/nuvselect/ws32pc/PHANGS_IR4_hst_wfc3_%s_v1p3_multi_assoc-nuvselect-ws32pc-main.fits' %(root_dir, galaxy_cat, galaxy_cat)
cluster_table_file_nuv_64pc = '%s/data_misc/multiscale_stellar_associations/%s/nuvselect/ws64pc/PHANGS_IR4_hst_wfc3_%s_v1p3_multi_assoc-nuvselect-ws64pc-main.fits' %(root_dir, galaxy_cat, galaxy_cat)

cluster_mask_file_v_08pc = '%s/data_misc/multiscale_stellar_associations/%s/vselect/ws8pc/PHANGS_IR4_hst_wfc3_%s_v1p3_multi_assoc-vselect-ws8pc-idmask.fits' %(root_dir, galaxy_cat, galaxy_cat)
cluster_mask_file_v_16pc = '%s/data_misc/multiscale_stellar_associations/%s/vselect/ws16pc/PHANGS_IR4_hst_wfc3_%s_v1p3_multi_assoc-vselect-ws16pc-idmask.fits' %(root_dir, galaxy_cat, galaxy_cat)
cluster_mask_file_v_32pc = '%s/data_misc/multiscale_stellar_associations/%s/vselect/ws32pc/PHANGS_IR4_hst_wfc3_%s_v1p3_multi_assoc-vselect-ws32pc-idmask.fits' %(root_dir, galaxy_cat, galaxy_cat)
cluster_mask_file_v_64pc = '%s/data_misc/multiscale_stellar_associations/%s/vselect/ws64pc/PHANGS_IR4_hst_wfc3_%s_v1p3_multi_assoc-vselect-ws64pc-idmask.fits' %(root_dir, galaxy_cat, galaxy_cat)

cluter_mask_file_nuv_08pc = '%s/data_misc/multiscale_stellar_associations/%s/nuvselect/ws8pc/PHANGS_IR4_hst_wfc3_%s_v1p3_multi_assoc-nuvselect-ws8pc-idmask.fits' %(root_dir, galaxy_cat, galaxy_cat)
cluter_mask_file_nuv_16pc = '%s/data_misc/multiscale_stellar_associations/%s/nuvselect/ws16pc/PHANGS_IR4_hst_wfc3_%s_v1p3_multi_assoc-nuvselect-ws16pc-idmask.fits' %(root_dir, galaxy_cat, galaxy_cat)
cluter_mask_file_nuv_32pc = '%s/data_misc/multiscale_stellar_associations/%s/nuvselect/ws32pc/PHANGS_IR4_hst_wfc3_%s_v1p3_multi_assoc-nuvselect-ws32pc-idmask.fits' %(root_dir, galaxy_cat, galaxy_cat)
cluter_mask_file_nuv_64pc = '%s/data_misc/multiscale_stellar_associations/%s/nuvselect/ws64pc/PHANGS_IR4_hst_wfc3_%s_v1p3_multi_assoc-nuvselect-ws64pc-idmask.fits' %(root_dir, galaxy_cat, galaxy_cat)

for prints in [cluster_table_file_v_08pc, cluster_table_file_v_16pc, cluster_table_file_v_32pc, cluster_table_file_v_64pc,
                cluster_table_file_nuv_08pc, cluster_table_file_nuv_16pc, cluster_table_file_nuv_32pc, cluster_table_file_nuv_64pc,
                cluster_mask_file_v_08pc, cluster_mask_file_v_16pc, cluster_mask_file_v_32pc, cluster_mask_file_v_64pc,
                cluter_mask_file_nuv_08pc, cluter_mask_file_nuv_16pc, cluter_mask_file_nuv_32pc, cluter_mask_file_nuv_64pc]:
    
    if os.path.isfile(prints):
        print('File found: %s' %prints)
    else: 
        print('File not found: %s' %prints)

# Read in the data...
props_clusters_v_08pc = QTable.read(cluster_table_file_v_08pc)
props_clusters_v_16pc = QTable.read(cluster_table_file_v_16pc)
props_clusters_v_32pc = QTable.read(cluster_table_file_v_32pc)
props_clusters_v_64pc = QTable.read(cluster_table_file_v_64pc)

props_clusters_nuv_08pc = QTable.read(cluster_table_file_nuv_08pc)
props_clusters_nuv_16pc = QTable.read(cluster_table_file_nuv_16pc)
props_clusters_nuv_32pc = QTable.read(cluster_table_file_nuv_32pc)
props_clusters_nuv_64pc = QTable.read(cluster_table_file_nuv_64pc)

hdu_cluster_mask_v_08pc = fits.open(cluster_mask_file_v_08pc)[0]
hdu_cluster_mask_v_16pc = fits.open(cluster_mask_file_v_16pc)[0]
hdu_cluster_mask_v_32pc = fits.open(cluster_mask_file_v_32pc)[0]
hdu_cluster_mask_v_64pc = fits.open(cluster_mask_file_v_64pc)[0]

hdu_cluster_mask_nuv_08pc = fits.open(cluter_mask_file_nuv_08pc)[0]
hdu_cluster_mask_nuv_16pc = fits.open(cluter_mask_file_nuv_16pc)[0]
hdu_cluster_mask_nuv_32pc = fits.open(cluter_mask_file_nuv_32pc)[0]
hdu_cluster_mask_nuv_64pc = fits.open(cluter_mask_file_nuv_64pc)[0]

File found: /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ic5332/hst_contsub/ic5332_hst_ha.fits
File found: /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ic5332/muse/IC5332_nebmask.fits
File found: /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ic5332/muse/IC5332-0.87asec_MAPS.fits
File found: /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ic5332/cutouts/sample.reg
File found: /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ic5332/cutouts/sample.pickel
File found: /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_misc/sample_table/phangs_sample_table_v1p6.fits
File found: /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_misc/Nebulae_catalogue_v3/Nebulae_catalogue_v3.fits
File found: /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ic5332/catalogue/ic5332_mask.fits
File found: /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_m

In [121]:
# Load regions, sample table and HDUs... 
hdus_cutouts  = cat_misc.load_pickle('%s/hdus_all.pickel' %cutout_dir)
regions       = cat_misc.load_pickle(regions_pickel_file)
props_all     = QTable.read('%s/props_all.fits' %dendro_dir)
hdu_catalog_mask = fits.open(catalog_mask_file)[0] # Neb catalogue mask

# Get only regions in the props table...
region_IDs = props_all['region_ID']
for key in regions.keys():
    regions[key] = regions[key][region_IDs.astype(int).tolist()]

[INFO] [load_pickle] Load /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ic5332/cutouts/hdus_all.pickel
[INFO] [load_pickle] Load /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ic5332/cutouts/sample.pickel


In [122]:
# Get cutouts... 
rerun_cutouts = False
if rerun_cutouts:

    # Get cutouts of the cluster mask
    hdus_cluster_mask_new_v_08pc = cat_cutouts.get_croppeddata_all(hdu_cluster_mask_v_08pc, regions) 
    hdus_cluster_mask_new_v_16pc = cat_cutouts.get_croppeddata_all(hdu_cluster_mask_v_16pc, regions)
    hdus_cluster_mask_new_v_32pc = cat_cutouts.get_croppeddata_all(hdu_cluster_mask_v_32pc, regions)
    hdus_cluster_mask_new_v_64pc = cat_cutouts.get_croppeddata_all(hdu_cluster_mask_v_64pc, regions)

    hdus_cluster_mask_new_nuv_08pc = cat_cutouts.get_croppeddata_all(hdu_cluster_mask_nuv_08pc, regions)
    hdus_cluster_mask_new_nuv_16pc = cat_cutouts.get_croppeddata_all(hdu_cluster_mask_nuv_16pc, regions)
    hdus_cluster_mask_new_nuv_32pc = cat_cutouts.get_croppeddata_all(hdu_cluster_mask_nuv_32pc, regions)
    hdus_cluster_mask_new_nuv_64pc = cat_cutouts.get_croppeddata_all(hdu_cluster_mask_nuv_64pc, regions)
    
    # Get cutouts of the neb catalog mask
    hdus_catalog_mask_new = cat_cutouts.get_croppeddata_all(hdu_catalog_mask, regions) 

    # Save cutouts...
    cat_misc.save_pickle(hdus_cluster_mask_new_v_08pc, '%s/hdus_cluster_mask_new_v_08pc.pickel' %cutout_dir)
    cat_misc.save_pickle(hdus_cluster_mask_new_v_16pc, '%s/hdus_cluster_mask_new_v_16pc.pickel' %cutout_dir)
    cat_misc.save_pickle(hdus_cluster_mask_new_v_32pc, '%s/hdus_cluster_mask_new_v_32pc.pickel' %cutout_dir)
    cat_misc.save_pickle(hdus_cluster_mask_new_v_64pc, '%s/hdus_cluster_mask_new_v_64pc.pickel' %cutout_dir)
    cat_misc.save_pickle(hdus_cluster_mask_new_nuv_08pc, '%s/hdus_cluster_mask_new_nuv_08pc.pickel' %cutout_dir)
    cat_misc.save_pickle(hdus_cluster_mask_new_nuv_16pc, '%s/hdus_cluster_mask_new_nuv_16pc.pickel' %cutout_dir)
    cat_misc.save_pickle(hdus_cluster_mask_new_nuv_32pc, '%s/hdus_cluster_mask_new_nuv_32pc.pickel' %cutout_dir)
    cat_misc.save_pickle(hdus_cluster_mask_new_nuv_64pc, '%s/hdus_cluster_mask_new_nuv_64pc.pickel' %cutout_dir)
    cat_misc.save_pickle(hdus_catalog_mask_new, '%s/hdus_catalog_mask_new.pickel' %cutout_dir)

else:

    # Load cutouts if already run...
    hdus_cluster_mask_new_v_08pc = cat_misc.load_pickle('%s/hdus_cluster_mask_new_v_08pc.pickel' %cutout_dir)
    hdus_cluster_mask_new_v_16pc = cat_misc.load_pickle('%s/hdus_cluster_mask_new_v_16pc.pickel' %cutout_dir)
    hdus_cluster_mask_new_v_32pc = cat_misc.load_pickle('%s/hdus_cluster_mask_new_v_32pc.pickel' %cutout_dir)
    hdus_cluster_mask_new_v_64pc = cat_misc.load_pickle('%s/hdus_cluster_mask_new_v_64pc.pickel' %cutout_dir)

    hdus_cluster_mask_new_nuv_08pc = cat_misc.load_pickle('%s/hdus_cluster_mask_new_nuv_08pc.pickel' %cutout_dir)
    hdus_cluster_mask_new_nuv_16pc = cat_misc.load_pickle('%s/hdus_cluster_mask_new_nuv_16pc.pickel' %cutout_dir)
    hdus_cluster_mask_new_nuv_32pc = cat_misc.load_pickle('%s/hdus_cluster_mask_new_nuv_32pc.pickel' %cutout_dir)
    hdus_cluster_mask_new_nuv_64pc = cat_misc.load_pickle('%s/hdus_cluster_mask_new_nuv_64pc.pickel' %cutout_dir)

    hdus_catalog_mask_new = cat_misc.load_pickle('%s/hdus_catalog_mask_new.pickel' %cutout_dir)

[INFO] [load_pickle] Load /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ic5332/cutouts/hdus_cluster_mask_new_v_08pc.pickel
[INFO] [load_pickle] Load /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ic5332/cutouts/hdus_cluster_mask_new_v_16pc.pickel
[INFO] [load_pickle] Load /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ic5332/cutouts/hdus_cluster_mask_new_v_32pc.pickel
[INFO] [load_pickle] Load /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ic5332/cutouts/hdus_cluster_mask_new_v_64pc.pickel
[INFO] [load_pickle] Load /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ic5332/cutouts/hdus_cluster_mask_new_nuv_08pc.pickel
[INFO] [load_pickle] Load /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ic5332/cutouts/hdus_cluster_mask_new_nuv_16pc.pickel
[INFO] [load_pickle] Load /Users/abarnes/Dropbox/work/

In [123]:
def get_all_clusters(region_ID, cluster_IDs, props_clusters, association_name):

    # If no cluster, fill with nans...
    if len(cluster_IDs) == 0: 
        props_clusters_new = cat_clusters.get_no_clusters(region_ID, props_clusters)

    # Only one cluster needs to be selected...
    if len(cluster_IDs) == 1:
        props_clusters_new = cat_clusters.get_one_clusters(region_ID, props_clusters, cluster_IDs)
        
    # If more than one cluster, select the one with the minimum age...
    elif len(cluster_IDs) > 1:  
        props_clusters_new = cat_clusters.get_multi_clusters(region_ID, props_clusters, cluster_IDs)


    colnames_old = ['reg_id', 
                'reg_ra', 
                'reg_dec', 
                'reg_area', 
                'reg_rad', 
                'reg_dolflux_Age_MinChiSq', 
                'reg_dolflux_Age_MinChiSq_err', 
                'reg_dolflux_Mass_MinChiSq', 
                'reg_dolflux_Mass_MinChiSq_err', 
                'reg_dolflux_Ebv_MinChiSq', 
                'reg_dolflux_Ebv_MinChiSq_err', 
                'reg_dolflux_Age_MinChiSq_ave', 
                'reg_dolflux_Mass_MinChiSq_sum', 
                'no_clusters', 
                'one_clusters', 
                'multiple_clusters', 
                'region_ID']

    colnames_new = ['region_ID_association_%s' %association_name, 
                    'ra_association_%s' %association_name, 
                    'dec_association_%s' %association_name, 
                    'area_association_%s' %association_name, 
                    'rad_association_%s' %association_name, 
                    'age_association_%s' %association_name, 
                    'age_err_association_%s' %association_name, 
                    'mass_association_%s' %association_name, 
                    'mass_err_association_%s' %association_name, 
                    'ebv_association_%s' %association_name, 
                    'ebv_err_association_%s' %association_name, 
                    'age_ave_association_%s' %association_name, 
                    'mass_sum_association_%s' %association_name, 
                    'no_clusters_association_%s' %association_name, 
                    'one_clusters_association_%s' %association_name, 
                    'multiple_clusters_association_%s' %association_name,
                    'region_ID']

    props_clusters_new = props_clusters_new[colnames_old]
    props_clusters_new.rename_columns(colnames_old, colnames_new)

    return props_clusters_new

In [127]:
region_ID_all = np.unique(hdu_catalog_mask.data)
props_clusters_all = []

for i in tqdm(range(len(props_all['region_ID']))):

    region_ID = props_all['region_ID'][i]

    # Get cluster IDs by masking the cluster mask with the region ID mask...
    cluster_IDs_v_08pc = cat_clusters.get_cluster_IDs(region_ID, i, hdus_catalog_mask_new, hdus_cluster_mask_new_v_08pc)
    cluster_IDs_v_16pc = cat_clusters.get_cluster_IDs(region_ID, i, hdus_catalog_mask_new, hdus_cluster_mask_new_v_16pc)
    cluster_IDs_v_32pc = cat_clusters.get_cluster_IDs(region_ID, i, hdus_catalog_mask_new, hdus_cluster_mask_new_v_32pc)
    cluster_IDs_v_64pc = cat_clusters.get_cluster_IDs(region_ID, i, hdus_catalog_mask_new, hdus_cluster_mask_new_v_64pc)

    cluster_IDs_nuv_08pc = cat_clusters.get_cluster_IDs(region_ID, i, hdus_catalog_mask_new, hdus_cluster_mask_new_nuv_08pc)
    cluster_IDs_nuv_16pc = cat_clusters.get_cluster_IDs(region_ID, i, hdus_catalog_mask_new, hdus_cluster_mask_new_nuv_16pc)
    cluster_IDs_nuv_32pc = cat_clusters.get_cluster_IDs(region_ID, i, hdus_catalog_mask_new, hdus_cluster_mask_new_nuv_32pc)
    cluster_IDs_nuv_64pc = cat_clusters.get_cluster_IDs(region_ID, i, hdus_catalog_mask_new, hdus_cluster_mask_new_nuv_64pc)

    # Get the cluster properties...
    props_clusters_new_v_08pc = get_all_clusters(region_ID, cluster_IDs_v_08pc, props_clusters_v_08pc, 'v_08pc')
    props_clusters_new_v_16pc = get_all_clusters(region_ID, cluster_IDs_v_16pc, props_clusters_v_16pc, 'v_16pc')
    props_clusters_new_v_32pc = get_all_clusters(region_ID, cluster_IDs_v_32pc, props_clusters_v_32pc, 'v_32pc')
    props_clusters_new_v_64pc = get_all_clusters(region_ID, cluster_IDs_v_64pc, props_clusters_v_64pc, 'v_64pc')

    props_clusters_new_nuv_08pc = get_all_clusters(region_ID, cluster_IDs_nuv_08pc, props_clusters_nuv_08pc, 'nuv_08pc')
    props_clusters_new_nuv_16pc = get_all_clusters(region_ID, cluster_IDs_nuv_16pc, props_clusters_nuv_16pc, 'nuv_16pc')
    props_clusters_new_nuv_32pc = get_all_clusters(region_ID, cluster_IDs_nuv_32pc, props_clusters_nuv_32pc, 'nuv_32pc')
    props_clusters_new_nuv_64pc = get_all_clusters(region_ID, cluster_IDs_nuv_64pc, props_clusters_nuv_64pc, 'nuv_64pc')

    # Join the tables together using region_ID - one row per region_ID...
    props_clusters_all_ = join(props_clusters_new_v_08pc, props_clusters_new_v_16pc, keys='region_ID')
    props_clusters_all_ = join(props_clusters_all_, props_clusters_new_v_32pc, keys='region_ID')
    props_clusters_all_ = join(props_clusters_all_, props_clusters_new_v_64pc, keys='region_ID')
    props_clusters_all_ = join(props_clusters_all_, props_clusters_new_nuv_08pc, keys='region_ID')
    props_clusters_all_ = join(props_clusters_all_, props_clusters_new_nuv_16pc, keys='region_ID')
    props_clusters_all_ = join(props_clusters_all_, props_clusters_new_nuv_32pc, keys='region_ID')
    props_clusters_all_ = join(props_clusters_all_, props_clusters_new_nuv_64pc, keys='region_ID')
    props_clusters_all += [props_clusters_all_]

100%|██████████| 135/135 [00:21<00:00,  6.36it/s]


In [128]:
props_clusters_stacked = vstack(props_clusters_all) # Stack all the regions together...
props_all_cluster = join(props_all, props_clusters_stacked, keys='region_ID') # Join the nebula table with the cluster table...

# Only take non-masked values in table
props_all_cluster.write('%s/props_all_cluster.fits' %dendro_dir, overwrite=True)
props_all_cluster

region_ID,x_max,y_max,x_com,y_com,ra_max,dec_max,ra_com,dec_com,npix,HA6562_FLUX_HST,HA6562_FLUX_ERR_HST,area_exact,radius_circ,flux_max,flux_min,flux_mean,radius_circ_pc,x_mom,y_mom,ra_mom,dec_mom,area_ellipse,major_sigma,minor_sigma,mean_sigma,position_angle,mean_sigma_pc,complexity_score,complexity_score_leaves,complexity_rms,complexity_std,flag_edge_hst,flag_touch_hst,gal_name,cen_x_MUSE,cen_y_MUSE,cen_ra_MUSE,cen_dec_MUSE,cen_x_geo_MUSE,cen_y_geo_MUSE,flag_edge_MUSE,flag_nearby_MUSE,flag_star_MUSE,deproj_dist_MUSE,deproj_phi_MUSE,region_area_MUSE,region_circ_rad_MUSE,HB4861_FLUX_MUSE,OIII5006_FLUX_MUSE,HA6562_FLUX_MUSE,NII6583_FLUX_MUSE,SII6716_FLUX_MUSE,SII6730_FLUX_MUSE,SIII9068_FLUX_MUSE,OI6300_FLUX_MUSE,NII5754_FLUX_MUSE,HEI5875_FLUX_MUSE,SIII6312_FLUX_MUSE,OI6363_FLUX_MUSE,OII7319_FLUX_MUSE,OII7330_FLUX_MUSE,HB4861_FLUX_ERR_MUSE,OIII5006_FLUX_ERR_MUSE,HA6562_FLUX_ERR_MUSE,NII6583_FLUX_ERR_MUSE,SII6716_FLUX_ERR_MUSE,SII6730_FLUX_ERR_MUSE,SIII9068_FLUX_ERR_MUSE,OI6300_FLUX_ERR_MUSE,NII5754_FLUX_ERR_MUSE,HEI5875_FLUX_ERR_MUSE,SIII6312_FLUX_ERR_MUSE,OI6363_FLUX_ERR_MUSE,OII7319_FLUX_ERR_MUSE,OII7330_FLUX_ERR_MUSE,HA6562_SIGMA_MUSE,NII6583_SIGMA_MUSE,OIII5006_SIGMA_MUSE,HA6562_VEL_MUSE,NII6583_VEL_MUSE,OIII5006_VEL_MUSE,HA6562_SIGMA_ERR_MUSE,NII6583_SIGMA_ERR_MUSE,OIII5006_SIGMA_ERR_MUSE,HA6562_VEL_ERR_MUSE,NII6583_VEL_ERR_MUSE,OIII5006_VEL_ERR_MUSE,HB4861_FLUX_CORR_MUSE,OIII5006_FLUX_CORR_MUSE,HA6562_FLUX_CORR_MUSE,NII6583_FLUX_CORR_MUSE,SII6716_FLUX_CORR_MUSE,SII6730_FLUX_CORR_MUSE,SIII9068_FLUX_CORR_MUSE,OI6300_FLUX_CORR_MUSE,NII5754_FLUX_CORR_MUSE,HEI5875_FLUX_CORR_MUSE,SIII6312_FLUX_CORR_MUSE,OI6363_FLUX_CORR_MUSE,OII7319_FLUX_CORR_MUSE,OII7330_FLUX_CORR_MUSE,HB4861_FLUX_CORR_ERR_MUSE,OIII5006_FLUX_CORR_ERR_MUSE,HA6562_FLUX_CORR_ERR_MUSE,NII6583_FLUX_CORR_ERR_MUSE,SII6716_FLUX_CORR_ERR_MUSE,SII6730_FLUX_CORR_ERR_MUSE,SIII9068_FLUX_CORR_ERR_MUSE,OI6300_FLUX_CORR_ERR_MUSE,NII5754_FLUX_CORR_ERR_MUSE,HEI5875_FLUX_CORR_ERR_MUSE,SIII6312_FLUX_CORR_ERR_MUSE,OI6363_FLUX_CORR_ERR_MUSE,OII7319_FLUX_CORR_ERR_MUSE,OII7330_FLUX_CORR_ERR_MUSE,EBV_MUSE,EBV_ERR_MUSE,met_scal_MUSE,met_scal_err_MUSE,logq_D91_MUSE,logq_D91_err_MUSE,t_N2_MUSE,t_S3_MUSE,t_N2_err_MUSE,t_S3_err_MUSE,BPT_NII_MUSE,BPT_SII_MUSE,BPT_OI_MUSE,LOG_SIGMA_SFR_CHAB_MUSE,LOG_SIGMA_STMASS_CHAB_MUSE,5thNN_dist_MUSE,NN_number_MUSE,Environment_MUSE,r_R25_MUSE,r_reff_MUSE,HA6562_LUMINOSITY_MUSE,Delta_met_scal_MUSE,pc_per_pixel_MUSE,EW_HA6562_raw_MUSE,EW_HA6562_raw_ERR_MUSE,EW_HB4861_raw_MUSE,EW_HB4861_raw_ERR_MUSE,EW_HA6562_fit_MUSE,EW_HA6562_fit_ERR_MUSE,EW_HB4861_fit_MUSE,EW_HB4861_fit_ERR_MUSE,EW_HA6562_bgcorr_MUSE,EW_HA6562_bgcorr_ERR_MUSE,EW_HB4861_bgcorr_MUSE,EW_HB4861_bgcorr_ERR_MUSE,HII_class_v2_MUSE,HII_class_v3_MUSE,HA6562_FLUX_CORR_HST,HA6562_FLUX_ERR_CORR_HST,HA6562_LUMINOSITY_HST,HA6562_LUMINOSITY_ERR_HST,region_circ_rad_pc_MUSE,region_ID_association_v_08pc,ra_association_v_08pc,dec_association_v_08pc,area_association_v_08pc,rad_association_v_08pc,age_association_v_08pc,age_err_association_v_08pc,mass_association_v_08pc,mass_err_association_v_08pc,ebv_association_v_08pc,ebv_err_association_v_08pc,age_ave_association_v_08pc,mass_sum_association_v_08pc,no_clusters_association_v_08pc,one_clusters_association_v_08pc,multiple_clusters_association_v_08pc,region_ID_association_v_16pc,ra_association_v_16pc,dec_association_v_16pc,area_association_v_16pc,rad_association_v_16pc,age_association_v_16pc,age_err_association_v_16pc,mass_association_v_16pc,mass_err_association_v_16pc,ebv_association_v_16pc,ebv_err_association_v_16pc,age_ave_association_v_16pc,mass_sum_association_v_16pc,no_clusters_association_v_16pc,one_clusters_association_v_16pc,multiple_clusters_association_v_16pc,region_ID_association_v_32pc,ra_association_v_32pc,dec_association_v_32pc,area_association_v_32pc,rad_association_v_32pc,age_association_v_32pc,age_err_association_v_32pc,mass_association_v_32pc,mass_err_association_v_32pc,ebv_association_v_32pc,ebv_err_association_v_32pc,age_ave_association_v_3